# Pre Post Analysis Of Opioid Shipment In Florida


In [2]:
# importing the required libraries
import pandas as pd
import numpy as np
import altair as alt
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [3]:
# loading data for drug prescriptions in Florida and control states
prescriptions = pd.read_parquet(
    r"C:\Users\annap\Downloads\prescriptions_fl (1).parquet"
)

In [4]:
prescriptions.head()


,BUYER_STATE,BUYER_COUNTY,CountyFIPS_x,StateFIPS_x,Year,MME,FIP_unique,County,State,Population,county_test,state_abbrev,CountyFIPS_y,StateFIPS_y,CountyName,_merge
index,,,,,,,,,,,,,,,,
527,FL,alachua,12001,12,2007.0,95259.627977,1200112,Alachua County,FL,242685.0,Alachua,FL,12001.0,12.0,Alachua,both
528,FL,alachua,12001,12,2008.0,114675.221922,1200112,Alachua County,FL,244888.0,Alachua,FL,12001.0,12.0,Alachua,both
529,FL,alachua,12001,12,2009.0,141281.048185,1200112,Alachua County,FL,246657.0,Alachua,FL,12001.0,12.0,Alachua,both
530,FL,alachua,12001,12,2010.0,150910.785876,1200112,Alachua County,FL,247614.0,Alachua,FL,12001.0,12.0,Alachua,both
531,FL,alachua,12001,12,2011.0,145750.680899,1200112,Alachua County,FL,249834.0,Alachua,FL,12001.0,12.0,Alachua,both


In [5]:
# reducing the number of columns in the drug prescriptions dataset by including only the relevant attributes to create a new dataset

prescriptions_reduced = prescriptions[
    ["Year", "MME", "State", "Population", "county_test"]
]

In [6]:
prescriptions_reduced.head(3)


,Year,MME,State,Population,county_test
index,,,,,
527,2007.0,95259.627977,FL,242685.0,Alachua
528,2008.0,114675.221922,FL,244888.0,Alachua
529,2009.0,141281.048185,FL,246657.0,Alachua


In [7]:
# removing unwanted index and checking the data again in the compact version of prescriptions dataset
prescriptions_reduced.reset_index(drop=True, inplace=True)
prescriptions_reduced

,Year,MME,State,Population,county_test
0,2007.0,95259.627977,FL,242685.0,Alachua
1,2008.0,114675.221922,FL,244888.0,Alachua
2,2009.0,141281.048185,FL,246657.0,Alachua
3,2010.0,150910.785876,FL,247614.0,Alachua
4,2011.0,145750.680899,FL,249834.0,Alachua
...,...,...,...,...,...
2214,2011.0,0.000000,NC,5798.0,Hyde
2215,2012.0,0.000000,NC,5689.0,Hyde
2216,2013.0,0.000000,NC,5654.0,Hyde
2217,2007.0,0.000000,FL,12357.0,Glades


In [8]:
# creating a copy of reduced dataset of prescriptions and converting some of the attributes to appropriate data type

prescriptions_reduced_copy = prescriptions_reduced.copy()

prescriptions_reduced_copy["Year"] = prescriptions_reduced_copy["Year"].astype("int64")
prescriptions_reduced_copy["Population"] = prescriptions_reduced_copy[
    "Population"
].astype("int64")

# renaming county_test to County for simplicity
prescriptions_reduced_copy.rename(
    columns={
        "county_test": "County",
    },
    inplace=True,
)


In [9]:
prescriptions_reduced_copy.head(3)


,Year,MME,State,Population,County
0,2007,95259.627977,FL,242685,Alachua
1,2008,114675.221922,FL,244888,Alachua
2,2009,141281.048185,FL,246657,Alachua


In [10]:
# creating a dataset that has all the drug prescriptions in the state of Florida

florida_prescriptions = prescriptions_reduced_copy[
    prescriptions_reduced_copy["State"] == "FL"
]

florida_prescriptions.head()

,Year,MME,State,Population,County
0,2007,95259.627977,FL,242685,Alachua
1,2008,114675.221922,FL,244888,Alachua
2,2009,141281.048185,FL,246657,Alachua
3,2010,150910.785876,FL,247614,Alachua
4,2011,145750.680899,FL,249834,Alachua


In [11]:
# Calculating opioid shipment per 100000 people in Florida
florida_prescriptions_copy = florida_prescriptions.copy()

florida_prescriptions_copy["Opioid_Shipment_Per_100K"] = (
    florida_prescriptions_copy["MME"] / florida_prescriptions_copy["Population"]
) * 100000

florida_prescriptions_copy.head(3)

,Year,MME,State,Population,County,Opioid_Shipment_Per_100K
0,2007,95259.627977,FL,242685,Alachua,39252.375704
1,2008,114675.221922,FL,244888,Alachua,46827.619941
2,2009,141281.048185,FL,246657,Alachua,57278.345308


In [12]:
florida_opioid_result = (
    florida_prescriptions_copy.groupby(["Year", "County"])["Opioid_Shipment_Per_100K"]
    .sum()
    .reset_index()
)
florida_opioid_result.head()


,Year,County,Opioid_Shipment_Per_100K
0,2007,Alachua,39252.375704
1,2007,Baker,52623.943931
2,2007,Bay,37047.299888
3,2007,Bradford,55003.869818
4,2007,Brevard,45672.967588


In [13]:
florida_opioid_result_yearwise = (
    florida_opioid_result.groupby("Year")["Opioid_Shipment_Per_100K"]
    .sum()
    .reset_index()
)
florida_opioid_result_yearwise.head()

,Year,Opioid_Shipment_Per_100K
0,2007,2.558877e+06
1,2008,3.111639e+06
2,2009,3.841404e+06
3,2010,4.894624e+06
4,2011,4.406423e+06


In [14]:
# split opioid shipment results of Florida into two subsets - before and after policy change

florida_before = florida_opioid_result[florida_opioid_result["Year"] < 2010]
florida_after = florida_opioid_result[florida_opioid_result["Year"] >= 2010]

In [15]:
# Function to plot a vertical line at year of policy implementation
def vertical_line(year):

    line = (
        alt.Chart(pd.DataFrame({"Year": [year], "color": ["black"]}))
        .mark_rule()
        .encode(x="Year:Q", color=alt.Color("color:N", scale=None))
    )

    return line

In [16]:
# creating the vertical line at 2010, the year of implementation of policy changes in Florida
line = vertical_line(2010)

In [17]:
# creating the regression model and calculating the error bands for creating the plots


def get_reg_fit_and_ci(data, color, xvar, yvar, legend, alpha=0.05):

    # Creating the grid for predicted values
    colour = color
    years = [2007, 2008, 2009, 2010, 2011, 2012, 2013]
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fitting the model and making the predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Utilizing the predictions to create the points and error bands in the chart
    predictions["Before/After"] = f"{legend}"
    reg = (
        alt.Chart(predictions)
        .mark_line()
        .encode(
            x=xvar,
            y=alt.Y(yvar),
            color=alt.Color(
                "Before/After", legend=alt.Legend(title="Before/After Policy Change")
            ),
        )
    )

    ci = (
        alt.Chart(predictions)
        .mark_errorband()
        .encode(
            alt.X(f"{xvar}:Q", axis=alt.Axis(format=".0f", values=years)),
            y=alt.Y(
                "ci_low",
                title="Opioid Shipment (MME) Per 100K Residents",
                scale=alt.Scale(zero=False),
            ),
            y2="ci_high",
            color=alt.value(f"{color}"),
        )
    )
    chart = ci + reg
    return predictions, chart

In [18]:
# Using the get_reg_fit_and_ci function to create the charts


def build_chart(data, color, xvar, yvar, legend, alpha=0.05):
    fit, reg_chart = get_reg_fit_and_ci(
        data=data,
        color=color,
        xvar=xvar,
        yvar=yvar,
        legend=legend,
        alpha=alpha,
    )
    return reg_chart

In [19]:
# creating the final plot for pre-post analysis of opioid shipments in Florida

florida_before_chart = build_chart(
    florida_before,
    "orange",
    "Year",
    "Opioid_Shipment_Per_100K",
    "Florida Before",
    alpha=0.05,
)

florida_after_chart = build_chart(
    florida_after,
    "blue",
    "Year",
    "Opioid_Shipment_Per_100K",
    "Florida After",
    alpha=0.05,
)

florida_final_pre_post_opioids = (
    florida_before_chart + florida_after_chart + line
).properties(title="Pre Post Analysis Of Opioid Shipment In Florida")

florida_final_pre_post_opioids

alt.LayerChart(...)